In [312]:
import numpy as np
import pandas as pd
import math
from sklearn.linear_model import ElasticNetCV
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

seed = 420
np.random.seed(seed)

In [313]:
def generate_data(dimention:int,data_len:int)->pd.DataFrame:
    variables = np.array(range(dimention),dtype=float)
    data = pd.DataFrame()
    for idx, variable in enumerate(variables):
        mu, sigma = variable, math.sqrt(variable + 1)
        generated = np.random.normal(mu,sigma,data_len)
        data[variable] = generated
    return data.copy()

def build_response(data:pd.DataFrame,colums_to_include:list,values:list,name = "y")->pd.Series:
    if np.shape(colums_to_include) != np.shape(values):
        raise ValueError('Columns and values have different shapes')
    response = np.random.normal(0,1,np.shape(data)[0])
    for i, column in enumerate(colums_to_include):
        response += data[column]*values[i]
    return pd.Series(name=name,data = response)

Il vero modello è definito come $y = 16\beta_5 + 8\beta_{10} + 4\beta_{15} + 2\beta_{20} + \varepsilon$.

> La $i$-esima variabile è campionata da $N ~ (i,\sqrt{i})$.

In [314]:
p = 71
data_len = 60 
toUse = [5,10,15,20]
values = [16,8,4,2]
B = 100

data = generate_data(dimention = p,
                     data_len = data_len)
y = build_response(data,toUse,values)
data["y"] = y
data.columns = data.columns.astype(str)

p_values = np.ones(shape=(B,p+1), dtype=float)

In [315]:
rng = np.random.default_rng(seed)

def sample_DataFrame(df:pd.DataFrame,sample_percentage_size:float)->pd.DataFrame:
    """"Funzione per campionare una percentuale dei dati dal DataSet."""
    if not (0.0 < sample_percentage_size < 1):
        raise AttributeError(f"sample_percentage_size must be in (0,1), got {sample_percentage_size} instead.")
    sample_size = int(df.shape[0] * sample_percentage_size)
    sample_idx = rng.integers(low=0,high=df.shape[0],size=sample_size)
    return df.iloc[sample_idx]

def split_DataFrame(df:pd.DataFrame, sample_percentage_size=0.5, response_name = "y")->tuple[pd.DataFrame,pd.DataFrame]:
    """Funzione per dividere il Dataset in 2 parti. Il primo Dataset ha elementi pari alla percentuale fornita."""
    y = df[response_name]
    df = df.drop(response_name, axis=1)
    
    df1 = sample_DataFrame(df,sample_percentage_size)
    df1[response_name] = y.iloc[df1.index]
    
    df2 = df.iloc[~df1.index]
    df2[response_name] = y.iloc[~df1.index]
    
    #print(f"First DataFrame has {df1.shape[0]} items, Second DataFrame has {df2.shape[0]} items.")
    return df1,df2

# Passi della simulazione:

**Ripeti B volte da 1 a 4**

1. Dividere il DataSet in due parti uguali $I_1$ e $I_2$
2. Applicare uno stimatore regolarizzato a $I_1$ ed estrarre i coefficienti diversi da zero
3. Applicare a $I_2$ un OLS con soli i parametri ottenuti da $I_2$
4. Definito $s = |\{\beta \ne 0, \forall \beta \in I_1\}|$ e $P_{raw,j}$ il j-esimo p-value
- Correggere i p-value ottenuti con $P_{corr,j} = min(P_{raw,j}*s,1)$
5. Aggregare i $P_{corr}$ della simulazione utilizzando:
- media
- mediana
- min-max

> Gli intervalli di confidenza possono essere aggregati con gli stessi metodi del punto 5.

La ripetizione (solitamente con $B = 50$ o $B = 100$) serve per avere dei risultati riproducibili indipendentemente dal seme usato.

In [316]:
def applyLasso(data:pd.DataFrame,response = "y")-> tuple[int,float]:
    X = data.drop(response,axis=1)
    y = data[response]
    lasso = ElasticNetCV(cv=5,random_state=seed,l1_ratio = 1)
    lasso.fit(X, y)
    print(f"Best alpha is: {lasso.alpha_}")
    return nonZeroCoeffiecients(X.columns,lasso.coef_)

def nonZeroCoeffiecients(dataColumns,regression_coefficients):
    coefficients = list(zip(dataColumns,regression_coefficients))
    coefficients_copy = coefficients.copy()
    for coefficient in coefficients_copy:
        if np.isclose(coefficient[1],0.0):
            coefficients.remove(coefficient)
    return coefficients

In [317]:
def useNonZeroCoefficient(data:pd.DataFrame,nonZeroCoefficients:tuple[str,float],response = "y"):
    coefficients_name = [coefficient[0] for coefficient in nonZeroCoefficients]
    X = data.drop(response,axis=1)
    y = data["y"]
    X = X[coefficients_name]
    return X, y

def applyOLS(data:pd.DataFrame,nonZeroCoefficients:tuple[str,float],response = "y"):
    X, y = useNonZeroCoefficient(data,nonZeroCoefficients)
    X = sm.add_constant(X)
    least_square = sm.OLS(y, X)
    least_square.fit()
    return least_square.fit()


In [318]:
def correct_pvalue(row,correction):
    return min(row*correction,1.0)

def updadte_pvalues(matrix,values,correction,simulation_number):
    pvalues_indexs = values.index.tolist()
    for index in pvalues_indexs:
        if index == "const":
            matrix[simulation_number][-1] = correct_pvalue(values[index],correction)
        else:
            matrix[simulation_number][int(index)] = correct_pvalue(values[index],correction)
    return matrix

In [319]:
for simulation_number in range(B):
    I_1, I_2 = split_DataFrame(data)
    coefficients = applyLasso(I_1)
    print(f"There are {len(coefficients)} in simulation {simulation_number}.")
    results = applyOLS(I_2,coefficients)
    p_values = updadte_pvalues(p_values,results.pvalues,len(coefficients),simulation_number)

p_values


Best alpha is: 75.94812095683231
There are 6 in simulation 0.
Best alpha is: 0.1328222850460807
There are 27 in simulation 1.
Best alpha is: 7.191139945917668
There are 17 in simulation 2.
Best alpha is: 151.95407593460732
There are 0 in simulation 3.
Best alpha is: 8.588405541427399
There are 19 in simulation 4.
Best alpha is: 30.03748186833125
There are 17 in simulation 5.
Best alpha is: 140.0256252660825
There are 0 in simulation 6.
Best alpha is: 60.87814048622095
There are 8 in simulation 7.
Best alpha is: 2.8104169866743103
There are 18 in simulation 8.
Best alpha is: 93.89156098162185
There are 3 in simulation 9.
Best alpha is: 13.498606750311646
There are 14 in simulation 10.
Best alpha is: 133.5442385548687
There are 0 in simulation 11.
Best alpha is: 153.5005619087772
There are 0 in simulation 12.
Best alpha is: 2.5387707018114862
There are 19 in simulation 13.
Best alpha is: 20.28302035149281
There are 17 in simulation 14.
Best alpha is: 0.13421943973469544
There are 26 in s

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 3.96232900e-01, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 3.48287129e-23, 4.87619576e-23],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [320]:
results.conf_int(alpha=.05)

,0,1
const,266.519369,298.014126


In [321]:
results.pvalues.index

Index(['const'], dtype='object')

In [322]:
# Aggiustare i pvalue in base al numero di elementi in S(I_1)
results.pvalues * len(coefficients)


const    0.0
dtype: float64